In [17]:
import pandas as pd, numpy as np
from functions import *
DATA_PATH = 'datasets'

In [18]:
def head(*args, n=3):
    for df in args:
        print("COLUMNS: ", df.shape[1], '\n', 'ROWS:    ', df.shape[0], sep="")
        display(df.head(n))

In [19]:
df_all = pd.read_csv(f'{DATA_PATH}/everything_grouped.csv').rename(columns={'school_dist':'dist'})
df_edu_dist = pd.read_csv(f'{DATA_PATH}/education_dist.csv').rename(columns={'school_dist':'dist'})

df_geo_dist = pd.read_csv(f'{DATA_PATH}/geo_county_school.csv')[[
    'county', 'dist', 'geo_county_point', 'geo_dist_point']]
df_geo_county = df_geo_dist.groupby(['county']).agg('first').reset_index()[['county', 'geo_county_point']]

df_all = df_geo_county.merge(df_all, on=['county']).rename(columns={'geo':'geo_county'})
df_edu_dist = df_geo_dist.merge(df_edu_dist, on=['county', 'dist'])

df_edu_dist = df_edu_dist[ ~ df_edu_dist.geo_dist.isna()]

df_all.insert(0, 'year', df_all.pop('year'))

head(df_all, df_edu_dist)

COLUMNS: 383
ROWS:    490


,year,county,geo_county_point,geo_county,CRIME_BASE_COUNT_cr_count,CRIME_BASE_COUNT_against_person,CRIME_BASE_COUNT_against_property,CRIME_BASE_COUNT_against_society,CRIME_BASE_COUNT_not_a_crime,CRIME_BASE_NORM_cr_rate,...,EDU_NORM_white_mobile_instances_rate,EDU_NORM_black_stable_rate,EDU_NORM_black_mobile_rate,EDU_NORM_black_mobile_instances_rate,EDU_NORM_hispanic_stable_rate,EDU_NORM_hispanic_mobile_rate,EDU_NORM_hispanic_mobile_instances_rate,EDU_NORM_asian_stable_rate,EDU_NORM_asian_mobile_rate,EDU_NORM_asian_mobile_instances_rate
0,2012,ADAMS,POINT (-104.1930918 39.8398269),MULTIPOLYGON (((-103.70574149517748 39.9999110...,35891,5041.0,23469.0,7381.0,0.0,7794.461287,...,32.45,54.66,45.08,47.91,67.49,32.71,36.4,78.55,21.37,23.54
1,2013,ADAMS,POINT (-104.1930918 39.8398269),MULTIPOLYGON (((-103.70574149517748 39.9999110...,33220,4593.0,23863.0,4763.0,1.0,7069.920277,...,32.45,54.66,45.08,47.91,67.49,32.71,36.4,78.55,21.37,23.54
2,2014,ADAMS,POINT (-104.1930918 39.8398269),MULTIPOLYGON (((-103.70574149517748 39.9999110...,34494,4975.0,24832.0,4686.0,1.0,7187.687538,...,32.45,54.66,45.08,47.91,67.49,32.71,36.4,78.55,21.37,23.54


COLUMNS: 142
ROWS:    175


,county,dist,geo_county_point,geo_dist_point,geo_county,geo_dist,pupil_total,stable,mobile,mobile_instances,...,white_mobile_instances_rate,black_stable_rate,black_mobile_rate,black_mobile_instances_rate,hispanic_stable_rate,hispanic_mobile_rate,hispanic_mobile_instances_rate,asian_stable_rate,asian_mobile_rate,asian_mobile_instances_rate
0,ADAMS,MAPLETON 1,POINT (-104.1930918 39.8398269),POINT (-104.9187196 39.8415103),MULTIPOLYGON (((-103.70574149517748 39.9999110...,MULTIPOLYGON (((-105.01581612299998 39.8144774...,9037,5077,3919,4133,...,52.08,48.04,51.40,53.63,60.31,39.19,42.19,47.22,52.78,53.70
1,ADAMS,ADAMS 12 FIVE STAR SCHOOLS,POINT (-104.1930918 39.8398269),POINT (-104.9668135 39.9262994),MULTIPOLYGON (((-103.70574149517748 39.9999110...,MULTIPOLYGON (((-105.05310614499996 39.9302934...,49889,34283,15424,16854,...,32.27,55.98,43.94,47.41,67.30,32.23,36.81,81.07,18.84,21.15
2,ADAMS,ADAMS COUNTY 14,POINT (-104.1930918 39.8398269),POINT (-104.9268419 39.8059605),MULTIPOLYGON (((-103.70574149517748 39.9999110...,MULTIPOLYGON (((-104.96883410999999 39.7910064...,8265,5510,3038,3397,...,47.45,48.20,51.80,53.60,68.67,35.20,39.57,80.00,20.00,20.00


In [20]:
for c in df_all.columns:
    if df_all[c].dtype == 'float64':
        if c.endswith("_rate"):
            if df_all[c].mean() > 1:
                df_all[c] = df_all[c].round(2)
            if df_all[c].mean() > 50:
                df_all[c] = df_all[c].round(0)


    words = c.split("_")
    new_words = words
    for w in words:
        if w.isupper():
            new_words = [new for new in new_words if new != w]
    col = "_".join(new_words)
    df_all = df_all.rename(columns={c: col})


df_all = df_all.loc[:, ~ df_all.columns.duplicated()]

In [21]:
df_all.to_csv(f'{DATA_PATH}/everything_grouped_ready.csv', index=False)

In [22]:
df_edu_dist.to_csv(f'{DATA_PATH}/education_dist_ready.csv', index=False)